In [ ]:
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
from aicsimageprocessing import read_ome_zarr
from aicsimageio import transforms, AICSImage
import skimage
import skimage.morphology 
import scipy
import numpy
import json

In [ ]:
def add_granularity(csvname):

    def measure_granularity(full_image,ProjType,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        for eachchan in channels.keys():
            image = full_image[0,eachchan,0,:,:]
            image = numpy.where(image<0,False,image)
            image = skimage.exposure.rescale_intensity(image, in_range='image', out_range=(0,1))
            for eachcompartment in compartments.keys():
                labels = full_image[0,eachcompartment,0,:,:]
                binary_labels = labels>0
                image = image*binary_labels
                startmean = numpy.mean(image)           

                #Calculate granularity
                pixels=image.copy()
                subfactor=.5
                samp_size=.25
                radius=5
                eros=15

                orig = image.copy()

                # Rescale image 
                new_shape = numpy.array(pixels.shape)
                new_shape = new_shape * subfactor
                i, j = (
                    numpy.mgrid[0 : new_shape[0], 0 : new_shape[1]].astype(float)
                    / subfactor
                )
                pixels = scipy.ndimage.map_coordinates(pixels, (i, j), order=1)

                mask = numpy.ones_like(pixels)
                prepixels = pixels.copy()

                # Do initial processing based on radius
                back_shape = new_shape * samp_size
                i, j = (
                    numpy.mgrid[0 : back_shape[0], 0 : back_shape[1]].astype(float)
                    / samp_size
                )
                back_pixels = scipy.ndimage.map_coordinates(pixels, (i, j), order=1)
                back_mask = (
                    scipy.ndimage.map_coordinates(mask.astype(float), (i, j)) > 0.9
                )

                # "Radius of structuring element"
                selem = skimage.morphology.disk(radius, dtype=bool)
                back_pixels_mask = numpy.zeros_like(back_pixels)
                back_pixels_mask[back_mask == True] = back_pixels[back_mask == True]
                back_pixels = skimage.morphology.erosion(back_pixels_mask, footprint=selem)
                back_pixels_mask = numpy.zeros_like(back_pixels)
                back_pixels_mask[back_mask == True] = back_pixels[back_mask == True]
                back_pixels = skimage.morphology.dilation(back_pixels_mask, footprint=selem)
                i, j = numpy.mgrid[0 : new_shape[0], 0 : new_shape[1]].astype(float)
                #
                # Make sure the mapping only references the index range of
                # back_pixels.
                #
                i *= float(back_shape[0] - 1) / float(new_shape[0] - 1)
                j *= float(back_shape[1] - 1) / float(new_shape[1] - 1)
                back_pixels = scipy.ndimage.map_coordinates(
                    back_pixels, (i, j), order=1
                )
                pixels -= back_pixels
                pixels[pixels < 0] = 0

                ero = pixels.copy()

                footprint = skimage.morphology.disk(1, dtype=bool)

                for eachcount in range(0,eros):
                    prev_ero = ero
                    eachcount +=1
                    ero_mask = numpy.zeros_like(ero)
                    ero_mask[mask == True] = ero[mask == True]
                    ero = skimage.morphology.erosion(ero_mask, footprint=footprint)

                    prevmean = numpy.mean(prev_ero)
                    currentmean = numpy.mean(ero)

                    gs = (prevmean - currentmean) * 100 / startmean
                    feature_name = f"Granularity_{eachcount}_{compartments[eachcompartment]}_{channels[eachchan]}_{ProjType}"

                    feature_dict[feature_name].append(gs)
                
        return feature_dict

    cols2D = ['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
    feature_dict={}
    channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachcount in range(1,16):
        for eachchan in channels.keys():
            for ProjType in cols2D:
                for eachcompartment in compartments.keys():
                    feature_dict[f"Granularity_{eachcount}_{compartments[eachcompartment]}_{channels[eachchan]}_{ProjType}"] = []
    
    count = 0
    df = pd.read_csv(csvname)
    for _, eachrow in df.iterrows():
        for ProjType in cols2D:
            im = AICSImage(eachrow[ProjType]).data
            feature_dict = measure_granularity(im,ProjType,feature_dict)
            count +=1 
            if count % 100 == 0:
                print(count)
                with open("granularity.json","w") as outfile:
                    json.dump(feature_dict, outfile)
    
    df = pd.concat([df,pd.DataFrame(feature_dict)],axis=1)
    df.to_csv(csvname,index=False)

In [ ]:
add_granularity('../data/mitocells-Copy2.csv')